# Three qubit quantum Fourier transform

In [1]:
import numpy as np

In [2]:
# single qubit gates
H = np.array([[1, 1], [1, -1]])/np.sqrt(2)
S = np.array([[1, 0], [0, 1j]])
T = np.array([[1, 0], [0, np.sqrt(1j)]])

In [3]:
# swap gate
swap = np.identity(4)[[0, 2, 1, 3]]
swap

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [4]:
def controlled_gate(U):
    """
    Controlled two-qubit gate, with the first qubit the control and the second the target.
    """
    cU = np.identity(4, dtype=U.dtype)
    cU[2:4, 2:4] = U
    return cU

In [5]:
# example
controlled_gate(H)

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678]])

In [6]:
def reorder_gate(G, perm):
    """
    Adapt gate 'G' to an ordering of the qubits as specified in 'perm'.
    
    Example, given G = np.kron(np.kron(A, B), C):
        reorder_gate(G, [1, 2, 0]) == np.kron(np.kron(B, C), A)
    """
    perm = list(perm)
    # number of qubits
    n = len(perm)
    # reorder both input and output dimensions
    perm2 = perm + [n + i for i in perm]
    return np.reshape(np.transpose(np.reshape(G, 2*n*[2]), perm2), (2**n, 2**n))

In [7]:
# consistency check

A = np.random.randn(2, 2)
B = np.random.randn(2, 2)
C = np.random.randn(2, 2)

Gtest = np.kron(np.kron(A, B), C)

# compare
np.linalg.norm(reorder_gate(Gtest, [1, 2, 0]) - np.kron(np.kron(B, C), A))

9.321907481344796e-16

In [8]:
# construct gates for each step as 8x8 matrices

# first Hadamard gate
G1 = np.kron(H, np.identity(4))
# first controlled-S gate
G2 = np.kron(reorder_gate(controlled_gate(S), [1, 0]), np.identity(2))
# controlled-T gate
G3 = reorder_gate(np.kron(controlled_gate(T), np.identity(2)), [1, 2, 0])
# second Hadamard gate
G4 = np.kron(np.kron(np.identity(2), H), np.identity(2))
# second controlled-S gate
G5 = np.kron(np.identity(2), reorder_gate(controlled_gate(S), [1, 0]))
# third Hadamard gate
G6 = np.kron(np.identity(4), H)
# final swap gate
G7 = reorder_gate(np.kron(swap, np.identity(2)), [0, 2, 1])

In [9]:
# compose all gates to obtain the matrix representation of the quantum Fourier transform circuit
UQFT = G7 @ G6 @ G5 @ G4 @ G3 @ G2 @ G1
UQFT.shape

(8, 8)

In [10]:
# Fourier matrix, as reference
UFref = np.array([[np.exp(2*np.pi*1j*j*k/8)/np.sqrt(8) for j in range(8)] for k in range(8)])

In [11]:
# compare (should be equal up to numerical rounding errors)
np.linalg.norm(UQFT - UFref)

2.5303720322343966e-15